<!-- TABS -->
# Create datatype

In [ ]:
# <tab: Vector>
from superduperdb import vector

db.add(vector)

In [ ]:
# <tab: Tensor>
from superduperdb.ext.torch import tensor

db.add()

In [ ]:
# <tab: Array>
...

In [ ]:
# <tab: Image>
from superduperdb.ext.pillow import pil_image

In [ ]:
# <tab: Audio>
...

In [ ]:
# <tab: Video>
...

In [ ]:
# <tab: Custom-in-DB>
...

In [ ]:
# <tab: Custom-Artifact>
...